# Surameria Results

Cargamos los resultados de iterar todos los conjuntos de indicadores

In [1]:
import pandas as pd
import numpy as np
import time
import plotly.plotly as py
import plotly.graph_objs as go

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = datos['CountryCode'].drop_duplicates().values.tolist()
pais_nombre = 'Suramerica'

preprocessing = 'imput'
year_init = 2012
year_range = 5
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']
conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

In [2]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [3]:
def buscar_mejores(df):
    return df[df.Resultados > 0.9].reset_index(drop=True)

In [4]:
def graficar_resultados(df,look_back):

    trace = go.Heatmap(z=df[df['Look Back'] == look_back]['%'],
                       x=df[df['Look Back'] == look_back]['Base'],
                       y=df[df['Look Back'] == look_back]['Target'],
                       zmax=1.0,
                       zmin=0.0,
                       colorscale = 'Viridis')
    data=[trace]
    
    layout = go.Layout(
        title=pais_nombre+' + '+search+': Total Heatmap with Look Back = '+str(look_back),
        xaxis=dict(title='Base',   titlefont=dict(size=18)),
        yaxis=dict(title='Target', titlefont=dict(size=18))
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename=pais_nombre+' + '+search+': Total Heatmap with Look Back = '+str(look_back))

In [5]:
def buscar_resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]
    df_temp = temp.replace("'","").replace("[","").replace("]","").replace(" ","").split('\n')
    results = []
    for i in df_temp: 
        results.append(float(i))
    
    # Códigos de los indicadores con registros 
    df_nombres = pd.DataFrame(conjunto_nombre, columns=['Resultados'])
    indicador_1 = conjunto[df_nombres[df_nombres['Resultados']==indicador_1].index.values[0]]
    indicador_2 = conjunto[df_nombres[df_nombres['Resultados']==indicador_2].index.values[0]]     
    df = pd.DataFrame(indicador_2)
    
    # Filtro indicadores repetidos
    if(indicador_1!=indicador_2):
        indicador_2 = np.array(df.loc[~df.ix[:,0].isin(indicador_1)]).flatten()
    df_zeros = tabla_base(indicador_2).dropna(how='all',axis=1)
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,results]),columns=['Indicadores','Resultados'])
    
    return df_results

In [6]:
def graficar_resultados_individuales(df,indicador_base,indicador_target,look_back):

    df_temp = buscar_resultados(df,indicador_base,indicador_target,look_back)

    data = [go.Bar(
                x= df_temp['Indicadores'],
                y= df_temp['Resultados'],

        marker=dict(     
            color = df_temp['Resultados'].values,
            colorscale ='Viridis',
            showscale = True
            ),
        )]

    layout = go.Layout(
        title=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back),
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig, filename=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back))

In [7]:
def conjuntos_performance(df,look_back,perform,predict,rango):
    return df[df['Look Back']==look_back].groupby(predict).mean().sort_values('%', ascending=perform).index.values[:rango]

In [8]:
def predecibles_predictores(df,look_back,rango):
    return pd.DataFrame(np.column_stack([conjuntos_performance(df,look_back,False,'Base',rango),conjuntos_performance(df,look_back,True,'Base',rango),conjuntos_performance(df,look_back,False,'Target',rango),conjuntos_performance(df,look_back,True,'Target',rango)]),columns=[['Predictor','Predictor','Predictable','Predictable'],['Best','Worse','Easy','Hard']])

# Original

In [9]:
original = pd.read_csv("suramerica_results_original.csv").drop(['Unnamed: 0'], 1)
search   = 'original'

## Mejores Combinaciones

In [10]:
original.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Privado,Ciencia,1,11,0.900546
1,Urbano,Ciencia,2,13,0.892362
2,Comercio,Ciencia,2,11,0.879526
3,Comercio,Ciencia,3,11,0.877208
4,Infraestructura,Ciencia,2,13,0.874702
5,Energia,Ciencia,1,13,0.862954
6,Ciencia,Ciencia,3,13,0.855968
7,Privado,Ciencia,2,11,0.855853
8,Urbano,Ciencia,1,13,0.842303
9,Comercio,Ciencia,1,11,0.840930


In [11]:
graficar_resultados_individuales(original,'Privado','Ciencia',1)

## Peores Combinaciones

In [12]:
original.sort_values('%', ascending=True).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Pobreza,Urbano,3,21,0.171713
1,Pobreza,Urbano,1,21,0.192287
2,Pobreza,Ayuda,2,69,0.195450
3,Pobreza,Salud,1,152,0.201384
4,Pobreza,Ayuda,3,69,0.208370
5,Pobreza,Urbano,2,21,0.210160
6,Pobreza,Salud,2,152,0.210652
7,Pobreza,Salud,3,152,0.215839
8,Pobreza,Ambiente,1,111,0.223511
9,Pobreza,Energia,3,53,0.225532


In [13]:
graficar_resultados_individuales(original,'Pobreza','Urbano',3)

## Resultados

In [14]:
graficar_resultados(original,1)

In [15]:
graficar_resultados(original,2)

In [16]:
graficar_resultados(original,3)

## Best and Worse Predictors and Predictable

- Con el objetivo de encontrar los conjuntos con mayor grado de predictibilidad, determinamos el promedio de su rendimiento para dos casos:

 **Predictor:** Conjunto que predice a otro conjunto
 
 **Predecible:** Conjunto que es predecido por otro conjunto

In [17]:
predecibles_predictores(original,1,5)

Predictor          Predictable         
          Best    Worse        Easy     Hard
0       Urbano  Pobreza     Ciencia    Ayuda
1  Agricultura  Ciencia     Publico    Salud
2        Clima    Ayuda      Urbano  Trabajo
3     Economia    Deuda    Economia   Genero
4      Privado  Trabajo       Clima    Deuda

# RandomizedSearchCV

In [18]:
random = pd.read_csv("suramerica_results_random.csv").drop(['Unnamed: 0'], 1)
search   = 'random'

## Mejores Combinaciones

In [19]:
random.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Comercio,Ciencia,1,11,0.924383
1,Comercio,Ciencia,2,11,0.920824
2,Comercio,Ciencia,3,11,0.903308
3,Ciencia,Ciencia,3,13,0.903255
4,Economia,Ciencia,3,11,0.895316
5,Economia,Ciencia,2,11,0.879495
6,Ciencia,Ciencia,1,13,0.873959
7,Clima,Ciencia,3,13,0.863778
8,Ambiente,Ciencia,3,13,0.859583
9,Agricultura,Urbano,2,24,0.855256


## Peores Combinaciones

In [20]:
random.sort_values('%', ascending=True).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Ciencia,Ayuda,1,71,0.323534
1,Ciencia,Trabajo,1,133,0.395530
2,Ciencia,Ayuda,3,71,0.400140
3,Ciencia,Pobreza,1,25,0.404672
4,Ciencia,Salud,1,152,0.414636
5,Ciencia,Ayuda,2,71,0.418732
6,Ciencia,Genero,1,177,0.421308
7,Ciencia,Trabajo,2,133,0.426061
8,Ciencia,Social,1,35,0.435801
9,Ciencia,Trabajo,3,133,0.443954


In [21]:
graficar_resultados(random,1)

In [22]:
graficar_resultados(random,2)

In [23]:
graficar_resultados(random,3)

## Best and Worse Predictors and Predictable

- Con el objetivo de encontrar los conjuntos con mayor grado de predictibilidad, determinamos el promedio de su rendimiento para dos casos:

 **Predictor:** Conjunto que predice a otro conjunto
 
 **Predecible:** Conjunto que es predecido por otro conjunto

In [24]:
predecibles_predictores(random,1,5)

Predictor            Predictable         
          Best      Worse        Easy     Hard
0  Agricultura    Ciencia     Ciencia    Ayuda
1     Economia      Ayuda     Publico    Salud
2        Clima  Educacion      Urbano  Trabajo
3     Comercio      Deuda    Economia   Genero
4     Ambiente   Ambiente       Clima    Deuda

## GridSearchCV

In [25]:
grid = pd.read_csv("suramerica_results_grid.csv").drop(['Unnamed: 0'], 1)
search   = 'grid'

## Mejores Combinaciones

In [26]:
random.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Comercio,Ciencia,1,11,0.924383
1,Comercio,Ciencia,2,11,0.920824
2,Comercio,Ciencia,3,11,0.903308
3,Ciencia,Ciencia,3,13,0.903255
4,Economia,Ciencia,3,11,0.895316
5,Economia,Ciencia,2,11,0.879495
6,Ciencia,Ciencia,1,13,0.873959
7,Clima,Ciencia,3,13,0.863778
8,Ambiente,Ciencia,3,13,0.859583
9,Agricultura,Urbano,2,24,0.855256


## Peores Combinaciones

In [27]:
random.sort_values('%', ascending=True).reset_index().drop(['index','Time','Results'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Ciencia,Ayuda,1,71,0.323534
1,Ciencia,Trabajo,1,133,0.395530
2,Ciencia,Ayuda,3,71,0.400140
3,Ciencia,Pobreza,1,25,0.404672
4,Ciencia,Salud,1,152,0.414636
5,Ciencia,Ayuda,2,71,0.418732
6,Ciencia,Genero,1,177,0.421308
7,Ciencia,Trabajo,2,133,0.426061
8,Ciencia,Social,1,35,0.435801
9,Ciencia,Trabajo,3,133,0.443954


In [28]:
graficar_resultados(grid,1)

In [29]:
graficar_resultados(grid,2)

In [30]:
graficar_resultados(grid,3)

## Best and Worse Predictors and Predictable

- Con el objetivo de encontrar los conjuntos con mayor grado de predictibilidad, determinamos el promedio de su rendimiento para dos casos:

 **Predictor:** Conjunto que predice a otro conjunto
 
 **Predecible:** Conjunto que es predecido por otro conjunto

In [31]:
predecibles_predictores(grid,1,5)

Predictor            Predictable         
          Best     Worse         Easy     Hard
0  Agricultura   Ciencia      Ciencia  Trabajo
1        Clima     Ayuda       Urbano    Ayuda
2     Comercio     Deuda      Publico    Salud
3     Ambiente  Ambiente     Economia   Genero
4        Deuda  Comercio  Agricultura    Deuda